In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
# from art.estimators.classification import PyTorchClassifier

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./RSLAD10.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    


        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./RSLAD10.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense_Label/WUSTL_Def9/y_pred_{attack_name}{eps}_Def9.npy", all_preds)


In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Initialize model, optimizer, and loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

def attack_pgd(model,train_batch_data,train_batch_labels,attack_iters=10,step_size=2/255.0,epsilon=8.0/255.0):
    ce_loss = torch.nn.CrossEntropyLoss().cuda()
    train_ifgsm_data = train_batch_data.detach() + torch.zeros_like(train_batch_data).uniform_(-epsilon,epsilon)
    train_ifgsm_data = torch.clamp(train_ifgsm_data,0,1)
    for i in range(attack_iters):
        train_ifgsm_data.requires_grad_()
        logits = model(train_ifgsm_data)
        loss = ce_loss(logits,train_batch_labels.cuda())
        loss.backward()
        train_grad = train_ifgsm_data.grad.detach()
        train_ifgsm_data = train_ifgsm_data + step_size*torch.sign(train_grad)
        train_ifgsm_data = torch.clamp(train_ifgsm_data.detach(),0,1)
        train_ifgsm_pert = train_ifgsm_data - train_batch_data
        train_ifgsm_pert = torch.clamp(train_ifgsm_pert,-epsilon,epsilon)
        train_ifgsm_data = train_batch_data + train_ifgsm_pert
        train_ifgsm_data = train_ifgsm_data.detach()
    return train_ifgsm_data

def rslad_inner_loss(model,
                teacher_logits,
                x_natural,
                y,
                optimizer,
                step_size=0.003,
                epsilon=0.031,
                perturb_steps=10,
                beta=6.0):
    # define KL-loss
    criterion_kl = nn.KLDivLoss(size_average=False,reduce=False)
    model.eval()
    batch_size = len(x_natural)
    # generate adversarial example
    x_adv = x_natural.detach() + 0.001 * torch.randn(x_natural.shape).cuda().detach()

    for _ in range(perturb_steps):
        x_adv.requires_grad_()
        with torch.enable_grad():
            loss_kl = criterion_kl(F.log_softmax(model(x_adv), dim=1),
                                       F.softmax(teacher_logits, dim=1))
            loss_kl = torch.sum(loss_kl)
        grad = torch.autograd.grad(loss_kl, [x_adv])[0]
        x_adv = x_adv.detach() + step_size * torch.sign(grad.detach())
        x_adv = torch.min(torch.max(x_adv, x_natural - epsilon), x_natural + epsilon)
        x_adv = torch.clamp(x_adv, 0.0, 1.0)

    model.train()

    x_adv = Variable(torch.clamp(x_adv, 0.0, 1.0), requires_grad=False)
    # zero gradient
    optimizer.zero_grad()
    logits = model(x_adv)
    return logits
  

In [12]:
import os
import argparse
import torch
# from rslad_loss import *
# from cifar10_models import *
import torchvision
from torchvision import datasets, transforms
import time
# # we fix the random seed to 0, in the same computer, this method can make the results same as before.
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# torch.backends.cudnn.deterministic = True

# prefix = 'mobilenet_v2-CIFAR10_RSLAD'
epochs = 10
batch_size = 128
epsilon = 8/255.0



student = DNNModel(input_shape,output_shape )
student = torch.nn.DataParallel(student)
student = student.cuda()
student.train()
optimizer = optim.SGD(student.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-4)
def kl_loss(a,b):
    loss = -a*b + torch.log(b+1e-5)*b
    return loss
teacher = DNNModel(input_shape,output_shape )
teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/dnn_pytorch.pt'))
teacher = torch.nn.DataParallel(teacher)
teacher = teacher.cuda()
teacher.eval()

/tmp/ipykernel_1298/63540866.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load('/home/jovyan/Sample_Based_Extension/WUSTL/transfer_atta

DataParallel(
  (module): DNNModel(
    (fc1): Linear(in_features=42, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=30, bias=True)
    (fc3): Linear(in_features=30, out_features=20, bias=True)
    (fc4): Linear(in_features=20, out_features=5, bias=True)
  )
)

In [13]:


# for epoch in range(0,epochs):
#     train_loss = 0.0
#     for step,(train_batch_data,train_batch_labels) in enumerate(train_loader):
#         student.train()
#         train_batch_data = train_batch_data.float().cuda()
#         train_batch_labels = train_batch_labels.cuda()
#         optimizer.zero_grad()
#         with torch.no_grad():
#             teacher_logits = teacher(train_batch_data)

#         adv_logits = rslad_inner_loss(student,teacher_logits,train_batch_data,train_batch_labels,optimizer,step_size=2/255.0,epsilon=epsilon,perturb_steps=3)
#         student.train()
#         nat_logits = student(train_batch_data)
#         kl_Loss1 = kl_loss(F.log_softmax(adv_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
#         kl_Loss2 = kl_loss(F.log_softmax(nat_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
#         kl_Loss1 = torch.mean(kl_Loss1)
#         kl_Loss2 = torch.mean(kl_Loss2)
#         loss = 5/6.0*kl_Loss1 + 1/6.0*kl_Loss2
#         loss.backward()
#         optimizer.step()
#         if step % 1000 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, step * len(train_batch_data), len(train_loader.dataset),
#                 100. * step / len(train_loader), loss.item()))
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
            
#             outputs = student(inputs)
#             kl_Loss1 = kl_loss(F.log_softmax(adv_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
#             kl_Loss2 = kl_loss(F.log_softmax(nat_logits,dim=1),F.softmax(teacher_logits.detach(),dim=1))
#             kl_Loss1 = torch.mean(kl_Loss1)
#             kl_Loss2 = torch.mean(kl_Loss2)
#             loss = 5/6.0*kl_Loss1 + 1/6.0*kl_Loss2
            
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
#     # print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")

    
#     # if (epoch%2 == 0):
#     #     test_accs = []
#     #     student.eval()
#     #     for step,(test_batch_data,test_batch_labels) in enumerate(test_loader):
#     #         test_ifgsm_data = attack_pgd(student,test_batch_data,test_batch_labels,attack_iters=20,step_size=0.003,epsilon=8.0/255.0)
#     #         logits = student(test_ifgsm_data)
#     #         predictions = np.argmax(logits.cpu().detach().numpy(),axis=1)
#     #         predictions = predictions - test_batch_labels.cpu().detach().numpy()
#     #         test_accs = test_accs + predictions.tolist()
#     #     test_accs = np.array(test_accs)
#     #     test_acc = np.sum(test_accs==0)/len(test_accs)
#     #     print('robust acc',np.sum(test_accs==0)/len(test_accs))
#     #     # torch.save(student.state_dict(),'./models/'+prefix+str(np.sum(test_accs==0)/len(test_accs))+'.pth')


#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

    

In [14]:

student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense/RSLAD/RSLAD_10.pt"))

/tmp/ipykernel_1298/628360739.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Defense

<All keys matched successfully>

In [15]:
# calculate_performance_metrics(x_test, y_test, student, 'DNN', 'baseline', '0')

In [16]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./RSLAD10.csv", mode='a', index=False, header=False)

In [17]:
epsilon = 0
Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB/WUSTL_Input100/x_test_adv_baseline_Def9.npy not found
start XGB 100 BIM
(1697, 42) (1697,)
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 FGSM
(490, 42) (490,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9673469387755103

macro
Precision: 0.4994692144373673
Recall: 0.49226804123711343
F1 Score: 0.49581589958158995

weighted
Precision: 0.9958577061397808
Recall: 0.9673469387755103
F1 Score: 0.9813935616087438

start XGB 100 PGD
(1697, 42) (1697,)
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 DF
(175, 42) (175,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8571428571428571

macro
Precision: 0.3333333333333333
Recall: 0.2857142857142857
F1 Score: 0.3076923076923077

weighted
Precision: 1.0
Recall: 0.8571428571428571
F1 Score: 0.9230769230769231

start XGB 100 AutoPGD
(3325, 42) (3325,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(520, 42) (520,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8576923076923076

macro
Precision: 0.3463203463203463
Recall: 0.6195090439276486
F1 Score: 0.3329596802224539

weighted
Precision: 0.9925324675324676
Recall: 0.8576923076923076
F1 Score: 0.9172049711237668

start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input20/x_test_adv_SINIFGSM_Def9.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input20/x_test_adv_VNIFGSM_Def9.npy not found
start RF 10 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input10/x_test_adv_baseline_Def9.npy not found
start RF 10 BIM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input10/x_test_adv_BIM_Def9.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF/WUSTL_Input10/x_test_adv_FGSM_Def9.npy not found
start RF 10 PGD
/home/jovyan/Sample_Based_Extension/WUST

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
epsilon = 0
Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input100/x_test_adv_baseline_Def9.npy not found
start XGB 100 BIM
(5847, 42) (5847,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.49820420728578757

macro
Precision: 0.15713896292822835
Recall: 0.20571780819792726
F1 Score: 0.16368502994148879

weighted
Precision: 0.3394102612597564
Recall: 0.49820420728578757
F1 Score: 0.3843869425519207

start XGB 100 FGSM
(2144, 42) (2144,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.23367537313432835

macro
Precision: 0.17434648684648685
Recall: 0.4919965113473993
F1 Score: 0.12677290628611085

weighted
Precision: 0.36118937809702734
Recall: 0.23367537313432835
F1 Score: 0.11825170692277072

start XGB 100 PGD
(5847, 42) (5847,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classifi

Accuracy: 0.9055555555555556

macro
Precision: 0.5
Recall: 0.4527777777777778
F1 Score: 0.4752186588921283

weighted
Precision: 1.0
Recall: 0.9055555555555556
F1 Score: 0.9504373177842566

start RF 50 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input50/x_test_adv_AutoPGD_Def9.npy not found
start RF 50 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input50/x_test_adv_ZOO_Def9.npy not found
start RF 50 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input50/x_test_adv_CaFA_Def9.npy not found
start RF 50 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input50/x_test_adv_SINIFGSM_Def9.npy not found
start RF 50 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input50/x_test_adv_VNIFGSM_Def9.npy not found
start RF 20 baseline
/home/j

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.961038961038961

macro
Precision: 0.5
Recall: 0.4805194805194805
F1 Score: 0.4900662251655629

weighted
Precision: 1.0
Recall: 0.961038961038961
F1 Score: 0.9801324503311258

start RF 20 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_AutoPGD_Def9.npy not found
start RF 20 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_ZOO_Def9.npy not found
start RF 20 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_CaFA_Def9.npy not found
start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_SINIFGSM_Def9.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input20/x_test_adv_VNIFGSM_Def9.npy not found
start RF 10 baseline
/home/jov

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
epsilon = 0
Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Exclude2Attack/WUSTL_Input100/x_test_adv_baseline_Def9.npy not found
start XGB 100 BIM
(2040, 42) (2040,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8955882352941177

macro
Precision: 0.19181102362204724
Recall: 0.18700102354145343
F1 Score: 0.18937548587717026

weighted
Precision: 0.9186243631310792
Recall: 0.8955882352941177
F1 Score: 0.9069600475588008

start XGB 100 FGSM
(329, 42) (329,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9452887537993921

macro
Precision: 0.33014861995753714
Recall: 0.32395833333333335
F1 Score: 0.3270241850683491

weighted
Precision: 0.9633515962286798
Recall: 0.9452887537993921
F1 Score: 0.9542347041508058

start XGB 100 PGD
(2040, 42) (2040,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8860103626943006

macro
Precision: 0.3333333333333333
Recall: 0.29533678756476683
F1 Score: 0.3131868131868132

weighted
Precision: 1.0
Recall: 0.8860103626943006
F1 Score: 0.9395604395604397

start RF 50 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input50/x_test_adv_AutoPGD_Def9.npy not found
start RF 50 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input50/x_test_adv_ZOO_Def9.npy not found
start RF 50 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input50/x_test_adv_CaFA_Def9.npy not found
start RF 50 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input50/x_test_adv_SINIFGSM_Def9.npy not found
start RF 50 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input50/x_test_adv_VNIFGSM_Def9.npy not fou

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8440860215053764

macro
Precision: 0.5
Recall: 0.4220430107526882
F1 Score: 0.4577259475218659

weighted
Precision: 1.0
Recall: 0.8440860215053764
F1 Score: 0.9154518950437318

start RF 20 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_AutoPGD_Def9.npy not found
start RF 20 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_ZOO_Def9.npy not found
start RF 20 CaFA
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_CaFA_Def9.npy not found
start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_SINIFGSM_Def9.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Exclude2Attack/WUSTL_Input20/x_test_adv_VNIFGSM_Def9.npy not found
start RF 10 b

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [20]:
epsilon = 0

Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_baseline_Def9.npy not found
start BIM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start FGSM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start PGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7297297297297297

macro
Precision: 0.33088235294117646
Recall: 0.24456521739130435
F1 Score: 0.28125

weighted
Precision: 0.987281399046105
Recall: 0.7297297297297297
F1 Score: 0.8391891891891892

start AutoPGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_ZOO_Def9.npy not found
start CaFA


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.28619119878603944

macro
Precision: 0.11685275984091274
Recall: 0.33304695304695303
F1 Score: 0.14236704258131722

weighted
Precision: 0.19097249355678012
Recall: 0.28619119878603944
F1 Score: 0.22308874530556064

start SINIFGSM
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_Euclidean/x_test_adv_VNIFGSM_Def9.npy not found
start baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_cosine/x_test_adv_baseline_Def9.npy not found
start BIM


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_baseline_Def9.npy not found
start BIM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start FGSM
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start PGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6880733944954128

macro
Precision: 0.3246753246753247
Recall: 0.2336448598130841
F1 Score: 0.2717391304347826

weighted
Precision: 0.956153937805314
Recall: 0.6880733944954128
F1 Score: 0.8002592740327085

start AutoPGD
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_ZOO_Def9.npy not found
start CaFA
Accuracy: 0.29456869009584663

macro
Precision: 0.16872153989801048
Recall: 0.34144454343978153
F1 Score: 0.14819391973448004

weighted
Precision: 0.34204517475049323
Recall: 0.29456869009584663
F1 Score: 0.2229133366896616

start SINIFGSM
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/Recommendation_manhattan/x_test_adv_VNIFGSM_Def9.npy not found

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

In [21]:
epsilon = 0
Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_Cluster/WUSTL_Input100/x_test_adv_baseline_Def9.npy not found
start XGB 100 BIM
(1697, 42) (1697,)
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 FGSM
(490, 42) (490,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9673469387755103

macro
Precision: 0.4994692144373673
Recall: 0.49226804123711343
F1 Score: 0.49581589958158995

weighted
Precision: 0.9958577061397808
Recall: 0.9673469387755103
F1 Score: 0.9813935616087438

start XGB 100 PGD
(1697, 42) (1697,)
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 DF
(175, 42) (175,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8571428571428571

macro
Precision: 0.3333333333333333
Recall: 0.2857142857142857
F1 Score: 0.3076923076923077

weighted
Precision: 1.0
Recall: 0.8571428571428571
F1 Score: 0.9230769230769231

start XGB 100 AutoPGD
(3325, 42) (3325,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.9015151515151515

macro
Precision: 0.3333333333333333
Recall: 0.3005050505050505
F1 Score: 0.3160690571049137

weighted
Precision: 1.0
Recall: 0.9015151515151515
F1 Score: 0.9482071713147411

start RF 10 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input10/x_test_adv_AutoPGD_Def9.npy not found
start RF 10 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input10/x_test_adv_ZOO_Def9.npy not found
start RF 10 CaFA
(286, 42) (286,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.8146853146853147

macro
Precision: 0.5
Recall: 0.40734265734265734
F1 Score: 0.4489402697495183

weighted
Precision: 1.0
Recall: 0.8146853146853147
F1 Score: 0.8978805394990366

start RF 10 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input10/x_test_adv_SINIFGSM_Def9.npy not found
start RF 10 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input10/x_test_adv_VNIFGSM_Def9.npy not found
start RF 1 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_baseline_Def9.npy not found
start RF 1 BIM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_BIM_Def9.npy not found
start RF 1 FGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_Cluster/WUSTL_Input1/x_test_adv_FGSM_Def9.npy not found
start RF 1 PGD
/home/jovyan/Sample_Based_Extension/

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

In [22]:
epsilon = 0
Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning/WUSTL_Input100/x_test_adv_baseline_Def9.npy not found
start XGB 100 BIM
(1697, 42) (1697,)
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 FGSM
(490, 42) (490,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9673469387755103

macro
Precision: 0.4994692144373673
Recall: 0.49226804123711343
F1 Score: 0.49581589958158995

weighted
Precision: 0.9958577061397808
Recall: 0.9673469387755103
F1 Score: 0.9813935616087438

start XGB 100 PGD
(1697, 42) (1697,)
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 DF
(175, 42) (175,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8571428571428571

macro
Precision: 0.3333333333333333
Recall: 0.2857142857142857
F1 Score: 0.3076923076923077

weighted
Precision: 1.0
Recall: 0.8571428571428571
F1 Score: 0.9230769230769231

start XGB 100 AutoPGD
(3325, 42) (3325,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(332, 42) (332,)
Accuracy: 0.8192771084337349

macro
Precision: 0.53125
Recall: 0.9085365853658536
F1 Score: 0.5084879589419661

weighted
Precision: 0.9887048192771084
Recall: 0.8192771084337349
F1 Score: 0.8899110060455008

start RF 50 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input50/x_test_adv_SINIFGSM_Def9.npy not found
start RF 50 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input50/x_test_adv_VNIFGSM_Def9.npy not found
start RF 20 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input20/x_test_adv_baseline_Def9.npy not found
start RF 20 BIM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input20/x_test_adv_BIM_Def9.npy not found
start RF 20 FGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input20/x_test_adv

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8181818181818182

macro
Precision: 0.5
Recall: 0.4090909090909091
F1 Score: 0.45

weighted
Precision: 1.0
Recall: 0.8181818181818182
F1 Score: 0.9

start RF 20 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input20/x_test_adv_SINIFGSM_Def9.npy not found
start RF 20 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input20/x_test_adv_VNIFGSM_Def9.npy not found
start RF 10 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input10/x_test_adv_baseline_Def9.npy not found
start RF 10 BIM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input10/x_test_adv_BIM_Def9.npy not found
start RF 10 FGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning/WUSTL_Input10/x_test_adv_FGSM_Def9.npy not found
start RF 10 PGD
/home/jovyan/Sample_Base

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
epsilon = 0

Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF" ]
active_learning_name = ["DensityWeighted", "BatchMode"]


base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ActiveLearning_DensityWeighted/WUSTL_Input100/x_test_adv_baseline_Def9.npy not found
start XGB 100 BIM
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 FGSM


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.9673469387755103

macro
Precision: 0.4994692144373673
Recall: 0.49226804123711343
F1 Score: 0.49581589958158995

weighted
Precision: 0.9958577061397808
Recall: 0.9673469387755103
F1 Score: 0.9813935616087438

start XGB 100 PGD
Accuracy: 0.5598114319387154

macro
Precision: 0.5568800031806993
Recall: 0.5196748930413992
F1 Score: 0.4278467694748347

weighted
Precision: 0.736023775272509
Recall: 0.5598114319387154
F1 Score: 0.49368667889456075

start XGB 100 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8571428571428571

macro
Precision: 0.3333333333333333
Recall: 0.2857142857142857
F1 Score: 0.3076923076923077

weighted
Precision: 1.0
Recall: 0.8571428571428571
F1 Score: 0.9230769230769231

start XGB 100 AutoPGD


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_

Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 1 PGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_PGD_Def9.npy not found
start RF 1 DF


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8068181818181818

macro
Precision: 0.3333333333333333
Recall: 0.2689393939393939
F1 Score: 0.2976939203354298

weighted
Precision: 1.0
Recall: 0.8068181818181818
F1 Score: 0.8930817610062894

start RF 1 AutoPGD
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_AutoPGD_Def9.npy not found
start RF 1 ZOO
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_ZOO_Def9.npy not found
start RF 1 CaFA
Accuracy: 0.8

macro
Precision: 0.20718922162221132
Recall: 0.3624190064794816
F1 Score: 0.19467630472916495

weighted
Precision: 0.9699267642186795
Recall: 0.8
F1 Score: 0.8722509680093736

start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/WUSTL_Input1/x_test_adv_SINIFGSM_Def9.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
epsilon = 0

Def = "Def9"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "10", "1"]
model_name = ["XGB", "RF"]

base_path = "/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data"
total_num_attack = ["EOAL", "EOAL_ExcludeCaFA", "EOAL_Exclude2Attack", "EOAL_Exclude3Attack"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{num_attack}/WUSTL_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{num_attack}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, student, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



In [15]:
# torch.save(student.state_dict(), "./RSLAD_10.pt")

In [17]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, student, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")